In [26]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf

import os
import sys
import time

from tensorflow import keras, Tensor, Variable, constant


In [4]:
def approximate_derivative(f, x, eps=1e-3):
    return (f(x+eps) - f(x)) / eps

f = lambda x: x**2

approximate_derivative(f, 5)



10.001000000002591

In [5]:
def g(x1, x2):
    return (x1 + 5) * (x2 ** 2)

def approximate_gradient(g, x1, x2, eps=1e-3):
    dg_dx1 = approximate_derivative(lambda x: g(x, x2), x1, eps)
    dg_dx2 = approximate_derivative(lambda x: g(x1, x), x2, eps)
    
    return dg_dx1, dg_dx2

print(approximate_gradient(g, 5, 4))

(16.000000000019554, 80.0100000000441)


In [24]:
x1: Variable = tf.Variable(2.0)
x2: Variable = tf.Variable(3.0)
    
with tf.GradientTape(persistent=True) as tape:
    z = g(x1, x2)
dz_dx1: Tensor = tape.gradient(z, x1)
dz_dx2: Tensor = tape.gradient(z, x2)
    
print(dz_dx1, dz_dx2)

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(42.0, shape=(), dtype=float32)


In [28]:
x1: constant = tf.constant(2.0)
x2: constant = tf.constant(3.0)
    
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x1)
    tape.watch(x2)
    z = g(x1, x2)
dz_dx1: Tensor = tape.gradient(z, x1)
dz_dx2: Tensor = tape.gradient(z, x2)
    
print(dz_dx1, dz_dx2)
del tape

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(42.0, shape=(), dtype=float32)


In [30]:
# 两个目标函数对一个变量求导数

x: Variable = tf.Variable(5.)
    
with tf.GradientTape() as tape:
    z1 = 3 * x
    z2 = x ** 2
    
tape.gradient([z1, z2], x)

<tf.Tensor: id=479, shape=(), dtype=float32, numpy=13.0>

In [33]:
# 求二阶导数

x1: Variable = tf.Variable(2.0)
x2: Variable = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as outer_tape:
    with tf.GradientTape(persistent=True) as inner_tape:
        z = g(x1, x2)
        
    inner_grads: list = inner_tape.gradient(z, [x1, x2])
    
outer_grads = [outer_tape.gradient(inner_grad, [x1, x2]) for inner_grad in inner_grads]
print(outer_grads)

del inner_tape, outer_tape

[[None, <tf.Tensor: id=601, shape=(), dtype=float32, numpy=6.0>], [<tf.Tensor: id=612, shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: id=610, shape=(), dtype=float32, numpy=14.0>]]


In [49]:
# 通过求导 实现梯度下降算法
learning_rate = 0.1 
x: Variable = tf.Variable(5.)

f = lambda x: x**2
    
for _ in range(1000):
    with tf.GradientTape() as tape:
        # target function
        z = f(x)
    dz_dx = tape.gradient(z, x)
    x.assign_sub(learning_rate * dz_dx)
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3e-45>


In [48]:
learning_rate = 0.1 
x: Variable = tf.Variable(5.)

f = lambda x: x**2
    
optimizer = keras.optimizers.SGD(lr=learning_rate)    
    
for _ in range(1000):
    with tf.GradientTape() as tape:
        # target function
        z = f(x)
    dz_dx = tape.gradient(z, x)
    optimizer.apply_gradients([(dz_dx, x)])
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3e-45>
